# Import libraries and define needed functions to process and insert data

In [1]:
import numpy as np
from io import StringIO
import datetime
import pandas as pd
import psycopg2
import time
#pip install ijson
import ijson
import json

In [2]:
#open the small (issues) file and load it into the memory
with open('gitIssues.json') as json_data:
    # use load() rather than loads() for JSON files
    record_list = json.load(json_data)

In [3]:
#function to convert list into readable array for postgres
def list_to_arr(lst):
    arr=str(lst).replace("[", "{")
    arr=arr.replace("]", "}")
    return arr

In [4]:
#function to convert epoch time into readable date for postgres
def date_to_pdate(epoch):
    my_time = time.strftime('%Y-%m-%d', time.localtime(epoch/1000))
    return my_time

In [5]:
# Connection params
dbname = "postgres"
User = "postgres"
#password = "Ananas65!"
password = "SMM695"
host='localhost'
port = "5432"

# Working with the gitissues file

In [6]:
#lets save columns names for later
columns=['assignees',
 'body',
 'closed_at',
 'closed_by',
 'comment_created_at',
 'comment_id',
 'comment_text',
 'comment_updated_at',
 'comment_user',
 'comment_user_id',
 'created_at',
 'labels',
 'n_comments',
 'project',
 'reactions',
 'repository',
 'state',
 'title',
 'updated_at',
 'user_id',
 'user_name']

## Create a table in Postgres 

In [7]:
# Let's create a table gitissues
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=User,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Print some info
    cursor.execute("SELECT current_user;")
    user_ = cursor.fetchone()

    cursor.execute("SELECT current_database();")
    db = cursor.fetchone()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    print("""
    Current user:     {}
    Current database: {}
    Version info:     {}
    """.format(user_[0], db[0], '-'.join(version[0].split()[:4])))
    
    cursor.execute("""CREATE TABLE public.gitissues (
        title varchar,
        state varchar,
        body varchar,
        user_name varchar,
        user_id varchar,
        closed_by varchar,
        comment_id varchar,
        comment_user_id varchar,
        comment_user varchar,
        comment_text varchar,
        project varchar, 
        closed_at date,
        created_at date,
        updated_at date,
        comment_created_at date,
        comment_updated_at date,
        n_comments int,
        labels text[],
        reactions text[],
        assignees text[]);""")
    # Commit changes
    connection.commit()

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()


    Current user:     postgres
    Current database: postgres
    Version info:     PostgreSQL-14.4-on-x86_64-apple-darwin20.6.0,
    


## Insert values into gitissues using pre-defined functions

In [8]:
# Let's insert the whole dataframe
start = time.time()
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=User,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Print some info
    cursor.execute("SELECT current_user;")
    user_ = cursor.fetchone()

    cursor.execute("SELECT current_database();")
    db = cursor.fetchone()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    print("""
    Current user:     {}
    Current database: {}
    Version info:     {}
    """.format(user_[0], db[0], '-'.join(version[0].split()[:4])))
    
    # SQL INSERT
    insert_sql = """INSERT INTO public.gitissues(title,state,body,user_name,user_id,closed_by,comment_id,comment_user_id,comment_user,comment_text,
               project,closed_at,created_at,updated_at,comment_created_at,
               comment_updated_at,n_comments,labels,reactions,assignees) 
               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    
    # Insert data
    for i in range(len(record_list)):
        epoch_closed=int(list(list(record_list[i]['closed_at'].values())[0].values())[0])
        closed_at=date_to_pdate(epoch_closed)
        epoch_created=int(list(list(record_list[i]['created_at'].values())[0].values())[0])
        created_at=date_to_pdate(epoch_created)
        epoch_upd=int(list(list(record_list[i]['updated_at'].values())[0].values())[0])
        updated_at=date_to_pdate(epoch_upd)
        epoch_comment_updated_at=int(list(list(record_list[i]['comment_updated_at'].values())[0].values())[0])
        comment_updated_at=date_to_pdate(epoch_comment_updated_at)
        epoch_comment_created_at=int(list(list(record_list[i]['comment_created_at'].values())[0].values())[0])
        comment_created_at=date_to_pdate(epoch_comment_created_at)

        assignees=list_to_arr(record_list[i]['assignees'])
        labels=list_to_arr(record_list[i]['labels'])
        reactions=list_to_arr(record_list[i]['reactions'])

        state=record_list[i]['state']
        user_name=record_list[i]['user']
        title=record_list[i]['title']
        body=record_list[i]['body']
        user_id=record_list[i]['user_id']
        n_comments=record_list[i]['n_comments']
        comment_id=record_list[i]['comment_id']
        comment_user_id=record_list[i]['comment_user_id']
        comment_user=record_list[i]['comment_user']
        comment_text=record_list[i]['comment_text']
        project=record_list[i]['project']
        try:
            closed_by=record_list[i]['closed_by']
        except:
            closed_by=None

        value=(title,state,body,user_name,user_id,closed_by,comment_id,comment_user_id,comment_user,comment_text,
               project,closed_at,created_at,updated_at,comment_created_at,
               comment_updated_at,n_comments,labels,reactions,assignees)
        cursor.execute(insert_sql, value)
        #print(cursor.statusmessage)

    # Commit changes
    connection.commit()

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()
#print the execution time
end = time.time()
print(end-start)


    Current user:     postgres
    Current database: postgres
    Version info:     PostgreSQL-14.4-on-x86_64-apple-darwin20.6.0,
    
5.180840015411377


## Split gitissues table into 2 tables containing info about comments and about issues. Set primary and foreign keys.

In [9]:
# Let's create a table gitissues
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=User,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Print some info
    cursor.execute("SELECT current_user;")
    user_ = cursor.fetchone()

    cursor.execute("SELECT current_database();")
    db = cursor.fetchone()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    print("""
    Current user:     {}
    Current database: {}
    Version info:     {}
    """.format(user_[0], db[0], '-'.join(version[0].split()[:4])))
    
    cursor.execute("""create sequence issue_id_seq start with 1;
create table issue as 
Select nextval('public."issue_id_seq"') as issue_id,title,
"state",body,user_name,user_id,closed_by,project,
closed_at,created_at,updated_at,n_comments,labels,reactions,assignees 
from ( 
Select distinct title,"state",body,user_name,user_id,closed_by,
               project,closed_at,created_at,updated_at,n_comments,labels,
               reactions,assignees from public.gitissues  )  as x;


create table comment as
        Select t.comment_user_id , t.comment_user ,t.comment_text,
t.comment_id,t.comment_created_at,t.comment_updated_at,tt.issue_id as fk_issue  from public.gitissues t 
       left  join public.issue tt 
         on t.title=tt.title and t.state=tt.state
        and coalesce(t.closed_by,'1')=coalesce(tt.closed_by,'1') and 
        t.closed_at=tt.closed_at and t.updated_at=tt.updated_at
        and t.n_comments = tt.n_comments and t.user_name=tt.user_name
        and t.user_id=tt.user_id and t.project =tt.project
        and t.created_at =tt.created_at and 
        coalesce(t.body,'1') =coalesce(tt.body,'1')
      and  coalesce(t.labels[1],'1') =coalesce(tt.labels[1],'1')
       and coalesce(t.labels[2],'1') =coalesce(tt.labels[2],'1') 
       and coalesce(t.labels[3],'1') =coalesce(tt.labels[3],'1');

        
create unique index issue_id_unique on public.issue (issue_id);
alter table issue add constraint 
issue_pk primary key using index issue_id_unique
;

ALTER TABLE public.comment
    ADD CONSTRAINT fk_issue 
    FOREIGN KEY (fk_issue) REFERENCES public.issue (issue_id);""")
    # Commit changes
    connection.commit()

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()


    Current user:     postgres
    Current database: postgres
    Version info:     PostgreSQL-14.4-on-x86_64-apple-darwin20.6.0,
    


# Working with the large (gitData object)

### Create a new table, containing deleted and added lines of codes and map them with gitData using sequence id. This step is needed because diff_parsed is stored as a nested dict.

In [10]:
# Let's create a table
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=User,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Print some info
    cursor.execute("SELECT current_user;")
    user_ = cursor.fetchone()

    cursor.execute("SELECT current_database();")
    db = cursor.fetchone()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    print("""
    Current user:     {}
    Current database: {}
    Version info:     {}
    """.format(user_[0], db[0], '-'.join(version[0].split()[:4])))
    
    cursor.execute("""CREATE TABLE public.added_deleted_lines (
    fk_id  bigint,
    type character varying,
    line_num bigint,
    line_str character varying);""")
    # Commit changes
    connection.commit()

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()


    Current user:     postgres
    Current database: postgres
    Version info:     PostgreSQL-14.4-on-x86_64-apple-darwin20.6.0,
    


### Inserting values into added_deleted_lines

In [11]:
#Lets unpack dicts and load them through the for loop
start = time.time()
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=User,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Print some info
    cursor.execute("SELECT current_user;")
    user_ = cursor.fetchone()

    cursor.execute("SELECT current_database();")
    db = cursor.fetchone()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    print("""
    Current user:     {}
    Current database: {}
    Version info:     {}
    """.format(user_[0], db[0], '-'.join(version[0].split()[:4])))
    
    # SQL INSERT
    insert_sql = """INSERT INTO public.added_deleted_lines(fk_id,type,line_num,line_str) 
               VALUES (%s,%s,%s,%s)"""
    
    # Insert data
    ################CHANGE RANGE TO THE WHOLE
    #set sequence id
    id_=0
    
    #go line by line without loading json into memory
    with open("gitData.json", "rb") as f:
        for record in ijson.items(f, "item"):
            id_+=1
            try:
                for key in record['diff_parsed'].keys():
                    line=record['diff_parsed'][key]
                    for i in range(len(line)):
                        line_num=line[i][0]
                        line_str=line[i][1]
                        type_=key
                        value=(id_,type_,line_num,line_str)
                        cursor.execute(insert_sql, value)
            except:
                pass
            #lets print each 1000 instert instead of all
            if id_%1000==0:
                print(cursor.statusmessage)

    # Commit changes
    connection.commit()

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()
end = time.time()
print(end-start)


    Current user:     postgres
    Current database: postgres
    Version info:     PostgreSQL-14.4-on-x86_64-apple-darwin20.6.0,
    
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
447.9036829471588


## Inserting all other columns except for diff_parsed

### Create a table gitdata and define all other 29 columns

In [22]:
# Let's create a table
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=User,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Print some info
    cursor.execute("SELECT current_user;")
    user_ = cursor.fetchone()

    cursor.execute("SELECT current_database();")
    db = cursor.fetchone()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    print("""
    Current user:     {}
    Current database: {}
    Version info:     {}
    """.format(user_[0], db[0], '-'.join(version[0].split()[:4])))
    
    cursor.execute("""CREATE TABLE public.gitData (
        id bigint,
        hash varchar,
        msg varchar,
        author_name varchar,
        committer_name varchar,
        author_date date,
        author_timezone bigint,
        committer_date date,
        committer_timezone bigint,
        branches text[],
        in_main_branch BOOLEAN,
        merge BOOLEAN,
        parents text[], 
        project_name varchar,
        deletions bigint,
        insertions bigint,
        lines bigint,
        files bigint,
        diff varchar,
        old_path varchar,
        new_path varchar,
        filename varchar,
        change_type varchar,
        added_lines bigint,
        deleted_lines bigint,
        source_code varchar,
        source_code_before varchar,
        nloc bigint,
        complexity int,
        token_count bigint);""")
    # Commit changes
    connection.commit()

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()


    Current user:     postgres
    Current database: postgres
    Version info:     PostgreSQL-14.4-on-x86_64-apple-darwin20.6.0,
    


### Instert data into the large gitData table

In [23]:
# Let's insert the whole dataframe
start = time.time()
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=User,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Print some info
    cursor.execute("SELECT current_user;")
    user_ = cursor.fetchone()

    cursor.execute("SELECT current_database();")
    db = cursor.fetchone()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    print("""
    Current user:     {}
    Current database: {}
    Version info:     {}
    """.format(user_[0], db[0], '-'.join(version[0].split()[:4])))
    
    # SQL INSERT
    insert_sql = """INSERT INTO public.gitData(
        id,
        hash ,
        msg ,
        author_name ,
        committer_name ,
        author_date ,
        author_timezone ,
        committer_date ,
        committer_timezone ,
        branches ,
        in_main_branch ,
        merge ,
        parents , 
        project_name ,
        deletions ,
        insertions ,
        lines ,
        files ,
        diff,
        old_path ,
        new_path ,
        filename ,
        change_type ,
        added_lines ,
        deleted_lines ,
        source_code ,
        source_code_before ,
        nloc ,
        complexity ,
        token_count) 
               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    
    # Insert data
    id_=0
    with open("gitData.json", "rb") as f:
        for record in ijson.items(f, "item"):
            id_+=1
            try:
                hash_=record['hash']
            except:
                hash_=None
            try:
                msg=record['msg']
            except:
                msg=None
            try:
                author_name=record['author_name']
            except:
                author_name=None
            try:
                committer_name =record['committer_name']
            except:
                committer_name=None
            try:
                author_date =date_to_pdate(int(list(list(record['author_date'].values())[0].values())[0]))
            except:
                author_date=None
            try:
                author_timezone =record['author_timezone']
            except:
                author_timezone=None
            try:
                committer_date=date_to_pdate(int(list(list(record['committer_date'].values())[0].values())[0]))
            except:
                committer_date=None
            try:
                committer_timezone =record['committer_timezone']
            except:
                committer_timezone=None
            try:
                branches=list_to_arr(record['branches'])
            except:
                branches=None
            try:
                in_main_branch=record['in_main_branch']
            except:
                in_main_branch=None
            try:
                merge =record['merge']
            except:
                merge=None
            try:
                parents =list_to_arr(record['parents'])
            except:
                parents=None
            try:
                project_name =record['project_name']
            except:
                project_name=None
            try:
                deletions =record['deletions']
            except:
                deletions=None
            try:
                insertions=record['insertions'] 
            except:
                insertions=None
            try:
                lines =record['lines']
            except:
                lines=None
            try:
                diff =record['diff']
            except:
                diff=None
            try:
                files =record['files']
            except:
                files=None
            try:
                old_path=record['old_path'] 
            except:
                old_path=None
            try:
                new_path =record['new_path']
            except:
                new_path=None
            try:
                filename =record['filename']
            except:
                filename=None
            try:
                change_type =record['change_type']
            except:
                change_type=None
            try:
                added_lines =record['added_lines']
            except:
                added_lines=None
            try:
                deleted_lines =record['deleted_lines']
            except:
                deleted_lines=None
            try:
                #otherwise we get an error null string \x00
                source_code=record['source_code'].replace('\x00', '') 
            except:
                source_code=None
            try:
                source_code_before=record['source_code_before'].replace('\x00', '')
            except:
                source_code_before=None
            try:
                nloc=record['nloc'] 
            except:
                nloc=None
            try:
                complexity=record['complexity'] 
            except:
                complexity=None
            try:
                token_count=record['token_count']
            except:
                token_count=None
        
            value=(
            id_,
            hash_,
            msg ,
            author_name ,
            committer_name ,
            author_date ,
            author_timezone ,
            committer_date ,
            committer_timezone ,
            branches ,
            in_main_branch ,
            merge ,
            parents , 
            project_name ,
            deletions ,
            insertions ,
            lines ,
            files ,
            diff,
            old_path ,
            new_path ,
            filename ,
            change_type ,
            added_lines ,
            deleted_lines ,
            source_code ,
            source_code_before ,
            nloc ,
            complexity ,
            token_count)
            cursor.execute(insert_sql, value)
            if id_%1000==0:
                print(cursor.statusmessage)

    # Commit changes
    connection.commit()

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()
end = time.time()
print(end-start)


    Current user:     postgres
    Current database: postgres
    Version info:     PostgreSQL-14.4-on-x86_64-apple-darwin20.6.0,
    
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
538.113312959671


### Divide the large gitData table into 2 - Commit_info and Update_info

#### commit_info

In [14]:
# Let's create a table gitissues
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=User,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Print some info
    cursor.execute("SELECT current_user;")
    user_ = cursor.fetchone()

    cursor.execute("SELECT current_database();")
    db = cursor.fetchone()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    print("""
    Current user:     {}
    Current database: {}
    Version info:     {}
    """.format(user_[0], db[0], '-'.join(version[0].split()[:4])))
    
    cursor.execute("""
    create table commit_info as select distinct 
        hash ,
        msg ,
        author_name ,
        committer_name ,
        author_date ,
        author_timezone ,
        committer_date ,
        committer_timezone ,
        branches ,
        in_main_branch ,
        merge ,
        parents , 
        project_name ,
        deletions ,
        insertions ,
        lines ,
        files
from public.gitData;""")
    # Commit changes
    connection.commit()

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()





    Current user:     postgres
    Current database: postgres
    Version info:     PostgreSQL-14.4-on-x86_64-apple-darwin20.6.0,
    


#### update_info

In [15]:
# run in 2 times, I don't know why but it worked only after 2 executions
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=User,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Print some info
    cursor.execute("SELECT current_user;")
    user_ = cursor.fetchone()

    cursor.execute("SELECT current_database();")
    db = cursor.fetchone()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    print("""
    Current user:     {}
    Current database: {}
    Version info:     {}
    """.format(user_[0], db[0], '-'.join(version[0].split()[:4])))
    
    cursor.execute("""
    create table update_info as select hash as fk_hash,"id",
    old_path ,
        new_path ,
        filename ,
        change_type ,
        added_lines ,
        deleted_lines ,
        source_code ,
        source_code_before ,
        nloc ,
        diff,
        complexity ,
        token_count
from public.gitData;""")
    # Commit changes
    connection.commit()

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()



    Current user:     postgres
    Current database: postgres
    Version info:     PostgreSQL-14.4-on-x86_64-apple-darwin20.6.0,
    


#### Modify commit_info and update_info by defining primary and foreign keys to connect them with added_deleted_lines

In [16]:
try:
    connection = psycopg2.connect(dbname=dbname,
                            user=User,
                            password=password,
                            host=host,
                            port=port)
    cursor = connection.cursor()

    # Print some info
    cursor.execute("SELECT current_user;")
    user_ = cursor.fetchone()

    cursor.execute("SELECT current_database();")
    db = cursor.fetchone()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()

    print("""
    Current user:     {}
    Current database: {}
    Version info:     {}
    """.format(user_[0], db[0], '-'.join(version[0].split()[:4])))
    
    cursor.execute("""
create unique index uniq_hash on public.commit_info (hash);
alter table public.commit_info  add constraint 
hash_pk primary key using index uniq_hash;

ALTER TABLE public.update_info
    ADD CONSTRAINT fk_hash
    FOREIGN KEY (fk_hash) REFERENCES public.commit_info (hash);
    
create unique index uniq_id on public.update_info (id);
alter table public.update_info  add constraint 
id_pk primary key using index uniq_id;


ALTER TABLE public.added_deleted_lines
    ADD CONSTRAINT fk_id
    FOREIGN KEY (fk_id) REFERENCES public.update_info (id);
    """)
    
    
    # Commit changes
    connection.commit()

except psycopg2.Error as error:
    print("Warning!", error)

finally:
        if connection is not None:
            cursor.close()
            connection.close()



    Current user:     postgres
    Current database: postgres
    Version info:     PostgreSQL-14.4-on-x86_64-apple-darwin20.6.0,
    


# Appendix: exploratory SQL queries

In [17]:
#double check if we imported data into update_info table correctly 
# on example of one of hashes
hash_=[]
with open("gitData.json", "rb") as f:
    for record in ijson.items(f, "item"):
        if record['hash']=='000b12b511b8c6c436720d2ab8fd9d6601ae263e':
            hash_.append(record)
            

In [18]:
#id 251
hash_[0]['diff_parsed']

{'added': [],
 'deleted': [[1356, '            define_value_at(/*index=*/{4});']]}

The record in added_deleted_lines with id 251 matches the original gitData values

## gitissues exploration

--Checking the relationships in the new tables
--1) Check if issues can have multiple titles
SELECT title, count(*)	FROM public.issue
group by title having count(*)>1;

--2) Let's have a look at the title that repeats 7 times
select * from public.issue where title ='Merge pull request #1 from tensorflow/master'

--3) observing the values in gitIssues where the title repears 11 times
select title,"state",user_name,user_id,closed_by,comment_id,
comment_user_id,comment_user,comment_text,
project,closed_at,created_at,updated_at,comment_created_at,
comment_updated_at,n_comments,labels,reactions,
assignees from public.gitissues where 
title='Feature Request: Add Pixel Unshuffle';

--4) check if we performed the join correctly and did not create new features
select count(*) from public.comment; 
--32329
select count(*) from public.gitissues; 
--32329 
select count(*) from public.comments where fk_issue is null;
--0
--32329 
select comment_id from public.gitissues
except
select comment_id from public.comment;  
--nulll
select comment_id from public.comments
except
select comment_id from public.gitissues; 
--null
--are there repetative comment ids?
Select count(*) , comment_id from comment 
group by comment_id having count(*) >1;


--De-bugging (performed before splitting gitissues into 2 tables)
Select * from public.gitissues where title in ( 
Select title from public.gitissues where comment_id = '901319756')
Select * from public.gitissues where body is null  and title = 'change efficientdet-lite0 --> efficientdet_lite0'

Select * from public.issue where title = 'Update setup.py';
--3 records with update setup in issues, 6 records in gitissues
--4 different body
--the reason of the 'bug' was our mistake to take only the first element in labels

## gitdata exploration

--check in original file fk_hash f016dddd612ca757879d3b75ef13892d1314832e
SELECT * FROM public.added_deleted_lines where fk_id =60690;
Select * from update_info where id = 60690;
Select * from (
Select fk_id, count(*) as cnt from added_deleted_lines 
group by fk_id ) as x
where cnt >1 order by cnt asc ;
--check nrows and compare to commit_info
select count(*) FROM public.gitdata;
SELECT count(*) FROM public.update_info;
--60996
SELECT count(*) FROM public.added_deleted_lines;
--3111566

select count(*) FROM public.gitdata;
--unique hash
SELECT hash,count(*) FROM public.commit_info 
group by hash
having count(hash)>1 ;

--did we miss any hash?
SELECT count(distinct hash) FROM public.gitdata;

SELECT count(*) FROM public.commit_info;
--15124

SELECT fk_id FROM public.added_deleted_lines 
where fk_id is null ;

SELECT count(distinct fk_id) FROM public.added_deleted_lines;
--59822
--number of unique ids
SELECT count(distinct "id")  FROM public.gitdata;
--60996

--how PG treats lists?
SELECT distinct branches FROM public.commit_info;

--are there any parents lists containing more than 1 value?
SELECT parents FROM public.commit_info where parents[2] is not Null;

-- are there many same messages?
SELECT msg,count(*) FROM public.commit_info 
group by msg
having count(msg)>1 ;


--check diff
select * from gitData where hash ='b0043072529b81276a69df29e00555333117646c';

--lets display one of the most repetative message
-- in this case it is one author
select * from public.commit_info where msg='Fix';

-- in this case - different authors
select * from public.commit_info where msg='Update README.md'; 

select diff, count(*) from public.update_info group by diff having count(*)>1;

--check the same diff
select * from update_info where diff like '%path: "tensorflow.lite.experimental.Ana%'

select * from update_info where fk_hash ='b0043072529b81276a69df29e00555333117646c'
order by diff ;

select fk_hash, count(*) over (partition by fk_hash order by fk_hash) as count_hash,
count(*) over (partition by fk_hash, diff order by fk_hash) as count_hash_diff
from public.update_info;

Select * from public.update_info where fk_hash = '000b12b511b8c6c436720d2ab8fd9d6601ae263e';
Select * from public.added_deleted_lines where fk_id = 251


select diff,fk_hash,count(*) from public.update_info
where diff is not null group by diff,fk_hash having count(*)>1;